<div style="display:flex;">
<img alt="New Relic" style="display:block;height:64px" src="https://raw.githubusercontent.com/dynovaio/open-source-office/refs/heads/master/assets/brands/new_relic.png" />
<img alt="SoftButterfly" style="display:block;height:64px;margin-left:auto" src="https://raw.githubusercontent.com/dynovaio/open-source-office/refs/heads/master/assets/dynova/dynova-logo-195x64.png" />
</div>

# New Relic Playground 3: Alerts

## Imports

Python Imports

In [ ]:
import os

Third-party libraries

In [ ]:
import dotenv
from newrelic_sb_sdk.client import NewRelicGqlClient
from newrelic_sb_sdk.graphql import nerdgraph
from newrelic_sb_sdk.graphql.enums import (
    AlertsIncidentPreference,
    AlertsNrqlConditionPriority,
    AlertsNrqlConditionTermsOperator,
    AlertsNrqlConditionThresholdOccurrences,
)
from newrelic_sb_sdk.graphql.input_objects import (
    AlertsNrqlConditionQueryInput,
    AlertsNrqlConditionStaticInput,
    AlertsNrqlConditionTermsInput,
    AlertsPolicyInput,
)
from newrelic_sb_sdk.graphql.objects import RootMutationType, RootQueryType
from newrelic_sb_sdk.utils.query import NULL_CURSOR
from newrelic_sb_sdk.utils.response import get_response_data, print_response
from sgqlc.operation import Operation

In [ ]:
AlertsNrqlConditionStaticInput

In [ ]:
AlertsNrqlConditionTermsInput

## Client setup

To setup the client, first we need to open load the credentials from environment variables.

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

NEW_RELIC_USER_KEY = os.environ.get("NEW_RELIC_USER_KEY", None)

if NEW_RELIC_USER_KEY is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

NEW_RELIC_ACCOUNT_ID = os.environ.get("NEW_RELIC_ACCOUNT_ID", None)

if NEW_RELIC_ACCOUNT_ID is None:
    raise ValueError("Environment variable NEW_RELIC_ACCOUNT_ID is not set.")

With environment varaibles loaded, we can proceed to instantiate the client

In [ ]:
newrelic = NewRelicGqlClient(new_relic_user_key=NEW_RELIC_USER_KEY)

## Client testing

In order to use an test the client we need configure `query_type` and `mutation_type` for the `nerdgraph` schema

In [ ]:
nerdgraph.query_type = RootQueryType
nerdgraph.mutation_type = RootMutationType

For testing we will use a simple query in GraphQL to get the atttributes from our user

```gql
query {
  actor {
    user {
      email
      id
      name
    }
  }
}
```

This query will be build from the `nerdgraph` schema.

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)

# Get the fields `id`, `email`, `name` from the `user` entity inside `actor`
operation.actor.user.__fields__("id", "email", "name")

This operation can be transformed into a GraphQL query

In [ ]:
query = operation.__to_graphql__()
print(query)

And this query is the one we send to be executed by our client.

In [ ]:
response = newrelic.execute(query)

The response data obtained is

In [ ]:
print_response(response)

We can also use the raw query directly written by hand. For this is recomendable to use the `build_query` method in order to get a clean query.

In [ ]:
query = newrelic.build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """
)

And execute directly, as in the previous execution, with the same result

In [ ]:
response = newrelic.execute(query)
print_response(response)

## Playground area

### Alerts management tasks with Nerdgraph

#### Example 01: List all available policies

Using a raw NRQL query we can list all available dashboards with this

In [ ]:
template = """
{
    actor {
        account(id: %(account_id)d) {
            alerts {
                policiesSearch(cursor: %(cursor)s) {
                    policies {
                        id
                        incidentPreference
                        name
                    }
                    totalCount
                    nextCursor
                }
            }
        }
    }
}
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
    "cursor": NULL_CURSOR,
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
policies_count = get_response_data(
    response, key_path="account:alerts:policiesSearch:totalCount", action="actor"
)
policies_count

In [ ]:
policies_list = get_response_data(
    response, key_path="account:alerts:policiesSearch:policies", action="actor"
)
len(policies_list)

In [ ]:
policies_list[0]

In [ ]:
next_cursor = get_response_data(
    response, key_path="account:alerts:policiesSearch:nextCursor", action="actor"
)
next_cursor

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)
alert_search_op = operation.actor.account(id=int(NEW_RELIC_ACCOUNT_ID))

alert_search_results_op = alert_search_op.alerts().policies_search(cursor=None)
alert_search_results_op.policies().__fields__("id", "incident_preference", "name")
alert_search_results_op.total_count()
alert_search_results_op.next_cursor()

operation

In [ ]:
response = newrelic.execute(operation.__to_graphql__())

In [ ]:
data = operation + response.json()

In [ ]:
data.actor.account.alerts.policies_search.total_count

In [ ]:
len(data.actor.account.alerts.policies_search.policies)

In [ ]:
data.actor.account.alerts.policies_search.policies[0]

In [ ]:
data.actor.account.alerts.policies_search.next_cursor

#### Example 02: Get a single policy

For getting a single policy is required know the identification this

In [ ]:
# Getting guid the result using a raw NRQL query
policy_id = policies_list[0]["id"]

In [ ]:
# Getting guid the result using Operation and nerdgraph
policy_id = data.actor.account.alerts.policies_search.policies[0]["id"]

In [ ]:
policy_id

Using a raw NRQL query we can getting a single policy with this

In [ ]:
template = """
{
    actor {
        account(id: %(account_id)d) {
            alerts {
                policy(id: %(policy_id)s) {
                    id
                    incidentPreference
                    name
                }
            }
        }
    }
}
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
    "policy_id": policy_id,
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
policy = get_response_data(response, key_path="account:alerts:policy", action="actor")
policy

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)
(
    operation.actor.account(id=int(NEW_RELIC_ACCOUNT_ID))
    .alerts()
    .policy(id=policy_id)
    .__fields__("id", "incident_preference", "name")
)
operation

In [ ]:
response = newrelic.execute(operation.__to_graphql__())

In [ ]:
data = operation + response.json()

In [ ]:
data.actor.account.alerts.policy

#### Example 03: Get all NRQL conditions

Using a raw NRQL query we can list all NRQL conditions with this

In [ ]:
# Getting guid the result using a raw NRQL query
policy_id = policies_list[4]["id"]

In [ ]:
# Getting guid the result using Operation and nerdgraph
policy_id = data.actor.account.alerts.policies_search.policies[4]["id"]

Using a raw NRQL query we can get all NRQL conditions with this

In [ ]:
template = """
{
    actor {
        account(id: %(account_id)d) {
            alerts {
                nrqlConditionsSearch {
                    nextCursor
                    totalCount
                    nrqlConditions {
                        id
                        name
                        nrql {
                            query
                        }
                        policyId
                    }
                }
            }
        }
    }
}
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
conditions_count = get_response_data(
    response,
    key_path="account:alerts:nrqlConditionsSearch:totalCount",
    action="actor",
)
conditions_count

In [ ]:
conditions_list = get_response_data(
    response,
    key_path="account:alerts:nrqlConditionsSearch:nrqlConditions",
    action="actor",
)
len(conditions_list)

In [ ]:
conditions_list[0]

In [ ]:
next_cursor = get_response_data(
    response,
    key_path="account:alerts:nrqlConditionsSearch:nextCursor",
    action="actor",
)
next_cursor

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)
alerts_conditions_search_op = (
    operation.actor.account(id=int(NEW_RELIC_ACCOUNT_ID))
    .alerts()
    .nrql_conditions_search()
)

alerts_conditions_search_op.__fields__("next_cursor", "total_count")

alerts_conditions_search_op.nrql_conditions().__fields__(
    "id", "name", "nrql", "policy_id"
)

operation

In [ ]:
response = newrelic.execute(operation.__to_graphql__())

In [ ]:
data = operation + response.json()

In [ ]:
data.actor.account.alerts.nrql_conditions_search.total_count

In [ ]:
len(data.actor.account.alerts.nrql_conditions_search.nrql_conditions)

In [ ]:
data.actor.account.alerts.nrql_conditions_search.nrql_conditions[0]

In [ ]:
data.actor.account.alerts.nrql_conditions_search.next_cursor

#### Example 04: Get conditions from a policy

For geeting a conditions from a policy is required know the identification this

In [ ]:
# Getting guid the result using a raw NRQL query
condition_id = conditions_list[0]["id"]

In [ ]:
# Getting guid the result using Operation and nerdgraph
condition_id = data.actor.account.alerts.nrql_conditions_search.nrql_conditions[0]["id"]

In [ ]:
condition_id

Using a raw NRQL query we can Get conditions from a policy with this

In [ ]:
template = """
{
    actor {
        account(id: %(account_id)d) {
            alerts {
                nrqlCondition(id: %(condition_id)s) {
                    description
                    enabled
                    id
                    name
                    nrql {
                        query
                    }
                    policyId
                    runbookUrl
                    type
                    terms {
                        operator
                        priority
                        threshold
                        thresholdDuration
                        thresholdOccurrences
                    }
                }
            }
        }
    }
}

"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
    "condition_id": condition_id,
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
condition = get_response_data(
    response,
    key_path="account:alerts:nrqlCondition",
    action="actor",
)
condition

In [ ]:
condition["policyId"]

In [ ]:
condition["nrql"]["query"]

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)

(
    operation.actor.account(id=int(NEW_RELIC_ACCOUNT_ID))
    .alerts()
    .nrql_condition(id=condition_id)
    .__fields__(
        "description",
        "enabled",
        "id",
        "name",
        "nrql",
        "policy_id",
        "runbook_url",
        "type",
        "terms",
    )
)

operation

In [ ]:
response = newrelic.execute(operation.__to_graphql__())

In [ ]:
data = operation + response.json()

In [ ]:
condition_op = data.actor.account.alerts.nrql_condition
condition_op

In [ ]:
condition_op["policy_id"]

In [ ]:
condition_op["nrql"]["query"]

#### Example 05: Create a policy

Using a raw NRQL query we can create a policy with this

In [ ]:
template = """
mutation {
    alertsPolicyCreate(
        accountId: %(account_id)d,
        policy: {
            incidentPreference: PER_POLICY,
            name: "Sample API Policy"
        }
    ) {
        accountId
        incidentPreference
        name
        id
    }
}
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
created_policy = get_response_data(
    response,
    action="alertsPolicyCreate",
)
created_policy

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `mutation_type` in `nerdgraph`
operation = Operation(nerdgraph.mutation_type)
policy_mutate_op = operation.alerts_policy_create(
    account_id=int(NEW_RELIC_ACCOUNT_ID),
    policy=AlertsPolicyInput(
        incident_preference=AlertsIncidentPreference.PER_POLICY,
        name="Sample API Policy Operation",
    ),
).__fields__(
    "account_id",
    "incident_preference",
    "name",
    "id",
)
operation

In [ ]:
response = newrelic.execute(operation.__to_graphql__())

In [ ]:
data = operation + response.json()

In [ ]:
data

#### Example 06: Append nrql condition to policy

For Append nrql condition to policy is required know the identification this

In [ ]:
# Getting policy_id the result using a raw NRQL query
policy_id_raw = created_policy["id"]

In [ ]:
# Getting policy_id the result using Operation and nerdgraph
policy_id_op = data.alerts_policy_create["id"]

Using a raw NRQL query we can append nrql condition to policy with this

In [ ]:
template = """
mutation {
    alertsNrqlConditionStaticCreate(
        accountId: %(account_id)d,
        policyId: %(policy_id)s,
        condition: {
            enabled: false,
            name: "Sample NRQL condition from API",
            nrql: {
                query: "SELECT average(duration) FROM Transaction"
            },
            terms: [
                {
                    operator: ABOVE,
                    priority: CRITICAL,
                    threshold: 10,
                    thresholdDuration: 300,
                    thresholdOccurrences: ALL
                }
            ]
        }
    ) {
        name
        policyId
        nrql {
            query
        }
        terms {
            operator
            threshold
            thresholdDuration
            thresholdOccurrences
            priority
        }
        id
    }
}
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
    "policy_id": policy_id_raw,
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
created_policy = get_response_data(
    response,
    action="alertsNrqlConditionStaticCreate",
)
created_policy

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `mutation_type` in `nerdgraph`
operation = Operation(nerdgraph.mutation_type)
nrql_create_mutate_op = operation.alerts_nrql_condition_static_create(
    account_id=NEW_RELIC_ACCOUNT_ID,
    policy_id=policy_id_op,
    condition=AlertsNrqlConditionStaticInput(
        enabled=False,
        name="Sample NRQL condition from API With Operation",
        nrql=AlertsNrqlConditionQueryInput(
            query="SELECT average(duration) FROM Transaction"
        ),
        terms=[
            AlertsNrqlConditionTermsInput(
                operator=AlertsNrqlConditionTermsOperator.ABOVE,
                priority=AlertsNrqlConditionPriority.CRITICAL,
                threshold=10,
                threshold_duration=300,
                threshold_occurrences=AlertsNrqlConditionThresholdOccurrences.ALL,
            )
        ],
    ),
)
(
    nrql_create_mutate_op.__fields__(
        "name",
        "policy_id",
        "nrql",
    )
)
(
    nrql_create_mutate_op.terms().__fields__(
        "operator",
        "threshold",
        "threshold_duration",
        "threshold_occurrences",
        "priority",
    )
)
nrql_create_mutate_op.id()

operation

In [ ]:
response = newrelic.execute(operation.__to_graphql__())

In [ ]:
response.json()